In [1]:
# import libraries

import os
from urllib.request import urlopen
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import time

# files saved under Data/ActiveTravelCounts
dir = 'Data/ActiveTravelCounts'
# raw files
loc_raw = '0-Count locations.csv'
central_raw = '2022-Central.csv'
inner_raw1 = '2022-Inner-Part1.csv'
inner_raw2 = '2022-Inner-Part2.csv'
outer_raw = '2022-Outer.csv'
# saved file name
location_fn = 'count_locations.geoparquet'
travelcounts_fn = 'travel_counts.parquet'

In [2]:
# reset if neccesary

counts_df = pd.read_parquet(os.path.join(dir, travelcounts_fn))

In [3]:
counts_df.shape

(2211584, 13)

In [4]:
# Peak Time - Monday to Friday (not on public holidays) between 06:30 and 09:30, and between 16:00 and 19:00
# According to https://tfl.gov.uk/fares/find-fares/tube-and-rail-fares
# Time -> datetime
counts_df['Time'] = pd.to_datetime(counts_df['Time'], format='%H:%M:%S').dt.time

In [5]:
counts_df[((counts_df.Time > time(6,30,00)) & (counts_df.Time < time(9,30,00))) | ((counts_df.Time > time(16,00,00)) & (counts_df.Time < time(19,00,00)))]

,Year,UnqID,Zone,datetime,Date,Weather,Time,Day,Round,Dir,Path,Mode,Count
3,2022,CENCY001,Central,2022-07-13 06:45:00,13/07/2022,Dry,06:45:00,Weekday,A,Northbound,Carriageway,Cargo bikes,0
4,2022,CENCY001,Central,2022-07-13 07:00:00,13/07/2022,Dry,07:00:00,Weekday,A,Northbound,Carriageway,Cargo bikes,0
5,2022,CENCY001,Central,2022-07-13 07:15:00,13/07/2022,Dry,07:15:00,Weekday,A,Northbound,Carriageway,Cargo bikes,0
6,2022,CENCY001,Central,2022-07-13 07:30:00,13/07/2022,Dry,07:30:00,Weekday,A,Northbound,Carriageway,Cargo bikes,0
7,2022,CENCY001,Central,2022-07-13 07:45:00,13/07/2022,Dry,07:45:00,Weekday,A,Northbound,Carriageway,Cargo bikes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
709871,2022,QWPCY126,Outer,2022-06-07 17:45:00,07/06/2022,Dry,17:45:00,Weekday,A,Westbound,Pavement - Westbound,Pedestrians,3
709872,2022,QWPCY126,Outer,2022-06-07 18:00:00,07/06/2022,Dry,18:00:00,Weekday,A,Westbound,Pavement - Westbound,Pedestrians,4
709873,2022,QWPCY126,Outer,2022-06-07 18:15:00,07/06/2022,Dry,18:15:00,Weekday,A,Westbound,Pavement - Westbound,Pedestrians,1
709874,2022,QWPCY126,Outer,2022-06-07 18:30:00,07/06/2022,Dry,18:30:00,Weekday,A,Westbound,Pavement - Westbound,Pedestrians,3


In [13]:

counts_df['ispeak'] = ((counts_df.Time > time(6,30,00)) & (counts_df.Time < time(9,30,00))) | ((counts_df.Time > time(16,00,00)) & (counts_df.Time < time(19,00,00)))
counts_df['ispeak'] = counts_df.ispeak.astype(int)

In [17]:
# drop modes in different classifications

cleaned_df = counts_df[(counts_df.Mode != 'Cycle hire bikes') & (counts_df.Mode != 'Cycle hire bikes')]
cleaned_df.shape

(2211328, 14)

In [18]:
cleaned_df.sample(5)

,Year,UnqID,Zone,datetime,Date,Weather,Time,Day,Round,Dir,Path,Mode,Count,ispeak
832182,2022,INNCY515,Inner,2022-06-14 19:30:00,14/06/2022,Dry,19:30:00,Weekday,A,Northbound,Pavement - Southbound,Cargo bikes,0,0
462819,2022,OUTCY273,Outer,2022-05-03 14:45:00,03/05/2022,Dry,14:45:00,Weekday,A,Eastbound,Pavement - Westbound,Pedestrians,2,0
145483,2022,OUTCY074,Outer,2022-06-28 08:45:00,28/06/2022,Dry,08:45:00,Weekday,A,Westbound,Pavement - Westbound,Conventional cycles,0,1
170706,2022,INNCY092,Inner,2022-05-25 10:30:00,25/05/2022,Dry,10:30:00,Weekday,A,Westbound,Pavement - Eastbound,Pedestrians,7,0
266308,2022,OUTCY148,Outer,2022-05-25 07:00:00,25/05/2022,Dry,07:00:00,Weekday,A,Northbound,Pavement - Northbound,Conventional cycles,0,1


In [9]:
summary_df = cleaned_df.groupby(['UnqID', 'Weather', 'Mode', 'ispeak']).Count.mean().reset_index()

/tmp/ipykernel_12067/703895083.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary_df = cleaned_df.groupby(['UnqID', 'Weather', 'Mode', 'ispeak']).Count.mean().reset_index()


In [10]:
summary_df[summary_df.Weather == 'Wet'].dropna()

,UnqID,Weather,Mode,ispeak,Count
36,CENCY002,Wet,Cargo bikes,False,0.260714
37,CENCY002,Wet,Cargo bikes,True,0.173333
38,CENCY002,Wet,Conventional cycles,False,6.317857
39,CENCY002,Wet,Conventional cycles,True,19.666667
42,CENCY002,Wet,E-scooters,False,0.092857
...,...,...,...,...,...
34191,QWPCY115,Wet,Conventional cycles,True,1.378788
34194,QWPCY115,Wet,E-scooters,False,0.119048
34195,QWPCY115,Wet,E-scooters,True,0.075758
34196,QWPCY115,Wet,Pedestrians,False,3.722222
